<a href="https://colab.research.google.com/github/elangbijak4/Adaptive-Stratified-Sampling---ASS/blob/main/ASS_Versi_Pilih_yang_apling_kuat_parameternya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# --- 1. SIMULASI POPULASI ---
# Mensimulasikan data: Strata 3 memiliki nilai yang sangat tinggi (Kelangkaan/Hotspot)
np.random.seed(42)
POP_SIZE = 1000
strata_data = {
    'Strata_1': np.random.normal(loc=10, scale=3, size=POP_SIZE), # Nilai Rendah
    'Strata_2': np.random.normal(loc=15, scale=4, size=POP_SIZE), # Nilai Sedang
    'Strata_3': np.random.normal(loc=80, scale=10, size=POP_SIZE) # Nilai EKSTREM/LANGKA
}
populasi = pd.DataFrame(strata_data)

print("--- ASS Berbasis Kelangkaan ---")
print(f"Rata-rata Populasi Sejati Strata 3: {populasi['Strata_3'].mean():.2f}")
print("-" * 40)

# --- 2. PENAMBILAN SAMPEL AWAL ---
N_initial = 10 # Sampel awal kecil (Pengintaian)
initial_samples = {}
initial_means = {}

print(f"Mengambil sampel awal N={N_initial} dari setiap strata...")
for strata, data in strata_data.items():
    sample = np.random.choice(data, size=N_initial, replace=False)
    initial_samples[strata] = sample
    initial_means[strata] = sample.mean()
    print(f"  Rata-rata Sampel Awal {strata}: {initial_means[strata]:.2f}")

# --- 3. PENGECKKAN KRITERIA (KELANGKAAN) ---
threshold = 30 # Ambang batas untuk mendefinisikan "nilai langka"
strata_to_adapt = []

print(f"\nMelakukan inspeksi: Kriteria Kelangkaan > {threshold}")
for strata, mean in initial_means.items():
    if mean > threshold:
        strata_to_adapt.append(strata)
        print(f"  ✅ {strata} terdeteksi sebagai HOTSPOT ({mean:.2f} > {threshold})")
    else:
        print(f"  ❌ {strata} berada di bawah ambang batas.")

# --- 4. ADAPTASI (MENINGKATKAN SAMPEL) ---
M_additional = 90 # Sampel tambahan
final_sample_sizes = {}
final_samples = {}
final_means = {}
final_std = {}

print(f"\nMelakukan Adaptasi (Pengambilan Tambahan M={M_additional})...")
for strata, data in strata_data.items():

    # Tentukan ukuran sampel akhir
    if strata in strata_to_adapt:
        final_size = N_initial + M_additional
    else:
        final_size = N_initial
    final_sample_sizes[strata] = final_size

    # Ambil sampel (menggabungkan yang awal dan tambahan)
    if strata in strata_to_adapt:
        remaining_data = [x for x in data if x not in initial_samples[strata]]
        additional_sample = np.random.choice(remaining_data, size=M_additional, replace=False)
        final_sample = np.concatenate([initial_samples[strata], additional_sample])
    else:
        final_sample = initial_samples[strata]

    final_samples[strata] = final_sample
    final_means[strata] = final_sample.mean()
    print(f"  {strata}: Sampel Akhir = {final_sample_sizes[strata]}. Rata-rata: {final_means[strata]:.2f}")

# --- 5. ESTIMASI AKHIR (Perbandingan) ---
# Estimasi Rata-rata Populasi Total (menggunakan Proporsi Strata yang sama)
populasi_size = POP_SIZE * 3 # Total 3000

# Estimasi Tanpa Adaptasi (hanya menggunakan N_initial)
est_no_adapt = sum(initial_means.values()) / 3

# Estimasi Dengan Adaptasi (menggunakan final_means)
est_with_adapt = sum(final_means.values()) / 3

true_mean = populasi.values.mean()

print("\n" + "=" * 40)
print(f"Rata-rata Populasi Sejati Total: {true_mean:.2f}")
print("-" * 40)
print(f"Estimasi TANPA Adaptasi (N=10 di semua strata): {est_no_adapt:.2f}")
print(f"Estimasi DENGAN Adaptasi (N+M di hotspot): {est_with_adapt:.2f}")
print("Hasil: Estimasi DENGAN Adaptasi jauh lebih akurat karena memfokuskan sumber daya ke area langka.")
print("=" * 40)

--- ASS Berbasis Kelangkaan ---
Rata-rata Populasi Sejati Strata 3: 80.06
----------------------------------------
Mengambil sampel awal N=10 dari setiap strata...
  Rata-rata Sampel Awal Strata_1: 11.46
  Rata-rata Sampel Awal Strata_2: 15.15
  Rata-rata Sampel Awal Strata_3: 76.55

Melakukan inspeksi: Kriteria Kelangkaan > 30
  ❌ Strata_1 berada di bawah ambang batas.
  ❌ Strata_2 berada di bawah ambang batas.
  ✅ Strata_3 terdeteksi sebagai HOTSPOT (76.55 > 30)

Melakukan Adaptasi (Pengambilan Tambahan M=90)...
  Strata_1: Sampel Akhir = 10. Rata-rata: 11.46
  Strata_2: Sampel Akhir = 10. Rata-rata: 15.15
  Strata_3: Sampel Akhir = 100. Rata-rata: 79.35

Rata-rata Populasi Sejati Total: 35.13
----------------------------------------
Estimasi TANPA Adaptasi (N=10 di semua strata): 34.39
Estimasi DENGAN Adaptasi (N+M di hotspot): 35.32
Hasil: Estimasi DENGAN Adaptasi jauh lebih akurat karena memfokuskan sumber daya ke area langka.
